---
### Demo of using Oracle Python Libraries to connect to local and OCI Database
---

##### Let's start with looking at using  cx_Oracle  library
##### -As of June 2022 this has been replaced by a new library called  oracledb.  See below for examples of using it


In [1]:
import cx_Oracle
#from ora_login import my_Oracle_login
#from ora_login import my_Oracle_login_docker
from ora_login import my_Oracle_login_cloud

#con = my_Oracle_login()
#con = my_Oracle_login_docker()
con = my_Oracle_login_cloud()

print(con)
cur = con.cursor()


<cx_Oracle.Connection to WHISKEY@student_high>


In [2]:
cur1 = con.cursor()
# execute a query returning the results to the cursor
cur1.execute('select table_name from user_tables')

# for each row returned to the cursor, print the record
print("Starting for loop")
for row in cur1:
      print(row)

Starting for loop
('WHISKIES_DATASET',)
('HOLIDAY',)
('STAGE',)
('DIRECTIONS',)


In [3]:
cur2=con.cursor()
cur2.execute('select * from whiskies_dataset')
for row in cur2:
    print(row)

(1, 'Aberfeldy', 2, 2, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2, 'PH152EB', 286580, 749680, 56.625191, -3.850199)
(2, 'Aberlour', 3, 3, 1, 0, 0, 4, 3, 2, 2, 3, 3, 2, 'AB389PJ', 326340, 842570, 57.467395, -3.229644)
(3, 'AnCnoc', 1, 3, 2, 0, 0, 2, 0, 0, 2, 2, 3, 2, 'AB55LI', 352960, 839320, 57.441751, -2.785295)
(4, 'Ardbeg', 4, 1, 4, 4, 0, 0, 2, 0, 1, 2, 1, 0, 'PA427EB', 141560, 646220, 55.640607, -6.108503)
(5, 'Ardmore', 2, 2, 2, 0, 0, 1, 1, 1, 2, 3, 1, 1, 'AB544NH', 355350, 829140, 57.350557, -2.743628)
(6, 'ArranIsleOf', 2, 3, 1, 1, 0, 1, 1, 1, 0, 1, 1, 2, 'KA278HJ', 194050, 649950, 55.699145, -5.278895)
(7, 'Auchentoshan', 0, 2, 0, 0, 0, 1, 1, 0, 2, 2, 3, 3, 'G814SJ', 247670, 672610, 55.922373, -4.439339)
(8, 'Auchroisk', 2, 3, 1, 0, 0, 2, 1, 2, 2, 2, 2, 1, 'AB553XS', 340754, 848623, 57.523872, -2.990865)
(9, 'Aultmore', 2, 2, 1, 0, 0, 1, 0, 0, 2, 2, 2, 2, 'AB553QY', 340754, 848623, 57.523872, -2.990865)
(10, 'Balblair', 2, 3, 2, 1, 0, 0, 2, 0, 2, 1, 2, 1, 'IV191LB', 270820, 885770, 57.842735

In [4]:
cur1.close()
cur2.close()
con.close()

In [5]:
#Let's now test cx_Oracle with a local DB on docker
p_username = "whiskey"
p_password = "WHISKEY"
p_host = "localhost"
p_service = "XEPDB1"
p_port = "1521"
    
con2 = cx_Oracle.connect(user=p_username, password=p_password, dsn="localhost/XEPDB1:1521")

In [6]:
cur3 = con2.cursor()
# execute a query returning the results to the cursor
cur3.execute('select table_name from user_tables')

# for each row returned to the cursor, print the record
print("Starting for loop")
for row in cur3:
      print(row)
        
cur3.close()
con2.close()

Starting for loop
('WHISKIES_DATASET',)
('HOLIDAY',)
('STAGE',)
('DIRECTIONS',)


---
### Now test using new library  oracledb
---

In [2]:
import oracledb

In [9]:
p_username = "whiskey"
p_password = "WHISKEY"
p_host = "localhost"
p_service = "XEPDB1"
p_dns = "localhost/XEPDB1"
p_port = "1521"

con = oracledb.connect(user=p_username, password=p_password, dsn=p_dns, port=p_port)
print('Do we have a good connection :', con.is_healthy())
print('Are we using a Thin connection :', con.thin)
print('Database Version :', con.version)

Do we have a good connection : True
Are we using a Thin connection : True
Database Version : 21.3.0.0.0


In [10]:
cur = con.cursor()
cur.execute('select table_name from user_tables')
for row in cur:
      print(row)
        

('WHISKIES_DATASET',)
('HOLIDAY',)
('STAGE',)
('DIRECTIONS',)


In [11]:
cur.close()
con.close()

In [12]:
#Alternative code with implicit closing of Cursor, after we are finished processing the data
import oracledb

con = oracledb.connect(user=p_username, password=p_password, dsn=p_dns, port=p_port)
with con.cursor() as cursor:
  for row in cursor.execute('select table_name from user_tables'):
      print(row)
        
print('Is the Connection open?', con.is_healthy())
print('What about the Cursor?', cur.connection)

con.close()

('WHISKIES_DATASET',)
('HOLIDAY',)
('STAGE',)
('DIRECTIONS',)
Is the Connection open? True
What about the Cursor? <oracledb.Connection disconnected>


---
### Connect to ADW using new oracledb
---

In [13]:
#Connection using Mutual TLS (mTLS) - Download Wallet file containing pem file
#Point the connection to the location of these Wallet files
import oracledb

p_username = "whiskey"
p_password = "22Highfield-_"

p_walletpass = '26Highfield-_'

con = oracledb.connect(user=p_username, password=p_password, dsn="student_high", 
                       config_dir="/Users/brendan.tierney/Dropbox/5-Database-Wallets/Wallet_student-Full",
                       wallet_location="/Users/brendan.tierney/Dropbox/5-Database-Wallets/Wallet_student-Full",
                       wallet_password=p_walletpass)
print(con)

with con.cursor() as cursor:
  for row in cursor.execute('select table_name from user_tables'):
      print(row)

print('Are we using a Thin connection :', con.thin)
con.close()

<oracledb.Connection to whiskey@student_high>
('WHISKIES_DATASET',)
('HOLIDAY',)
('STAGE',)
('DIRECTIONS',)
Are we using a Thin connection : True


In [2]:
#Thick connection using oracle client
#Warning: Once you enable Thick client, all connections in App will try to use Thick Client.
#         Only use Thick client when required in an App.  All other code/Apps should remove all references to it.
import oracledb

p_username = "whiskey"
p_password = "22Highfield-_"

oracledb.init_oracle_client(config_dir="/Applications/instantclient_19_8/network/admin")

con = oracledb.connect(user=p_username, password=p_password, dsn="student_high")
print(con)

with con.cursor() as cursor:
  for row in cursor.execute('select table_name from user_tables'):
      print(row)

print('Are we using a Thin connection :', con.thin)
con.close()

<oracledb.Connection to whiskey@student_high>
('WHISKIES_DATASET',)
('HOLIDAY',)
('STAGE',)
('DIRECTIONS',)
Are we using a Thin connection : False
